In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import LineString

import road

In [140]:
bbox =(48.7861400822045,2.197526708826558,48.93467868121917,2.4458831002388592)


highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                "secondary", "secondary_link", "tertiary", "tertiary_link", "residential"]

In [141]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query ="""
[out:json][timeout:180];
(
"""
overpass_query += ''.join([f'way["highway"="{highway}"]{bbox};\n' for highway in highway_list])
overpass_query +=""" 
);
out body;
>;
out skel qt;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [142]:
{'overpass_query': overpass_query,'tags': ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']}

{'overpass_query': '\n[out:json][timeout:180];\n(\nway["highway"="motorway"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="motorway_link"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="trunk"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="trunk_link"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="primary"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="primary_link"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="secondary"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="secondary_link"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\nway["highway"="tertiary"](48.7861400822045, 2.197526708826558, 48.93467868121917, 2.4458831002388592);\

In [143]:
"""
[out:json][timeout:180][bbox:{{bbox}}];
(
way["highway"="residential"]; // find all nodes in current bounding box with start_date tag
);
convert way 
    ::id = id(),
	"nodes" = geometry, // Should be implemened in futur version
	"oneway" = t["oneway"];
out body;
>;
out skel qt;
"""

'\n[out:json][timeout:180][bbox:{{bbox}}];\n(\nway["highway"="residential"]; // find all nodes in current bounding box with start_date tag\n);\nconvert way \n    ::id = id(),\n\t"nodes" = geometry, // Should be implemened in futur version\n\t"oneway" = t["oneway"];\nout body;\n>;\nout skel qt;\n'

overpass_url = "http://overpass-api.de/api/interpreter"
data = []
for highway in highway_list:
    overpass_query ="""
    [out:json][timeout:180];
    (
    """
    overpass_query += ''.join(f'way["highway"={highway}]{bbox};\n')
    overpass_query +=""" 
    );
    out body;
    >;
    out skel qt;
    """
    data.append(overpass_query)


import asyncio
import aiohttp
import time

async def get(url, data, session):
    try:
        async with session.get(url=url, data=data) as response:
            resp = await response.json()
            print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
            return await response
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(url, data):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, d, session) for d in data])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    return ret

response = await main(overpass_url, data)

In [144]:
way = pd.DataFrame([d for d in data['elements'] if d['type'] == 'way']).set_index('id')
nodes = pd.DataFrame([d for d in data['elements'] if d['type'] == 'node']).set_index('id')

way_exploded = way.explode('nodes').merge(nodes[['lat','lon']], left_on='nodes', right_index=True, how='left')

geom = way_exploded.groupby('id')[['lon', 'lat']].apply(lambda x: LineString(x.values))
geom.name = 'geometry'
way = gpd.GeoDataFrame(way.join(geom), crs=4326)


/home/simon/anaconda3/envs/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [145]:
tags = pd.DataFrame.from_records(way['tags'].values, index=way['tags'].index)
cols = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']

In [146]:
way_tags = way.drop(columns=['nodes', 'tags'], errors='ignore').join(tags[cols])

In [147]:
# OSMNX add edges speed, length and time
# garder les liens fortement connexes
# retirer les noeuds de degree 2

In [ ]:
# SOME CLEANING ON THE ONEWAY ... Work In Progress
way_tags['oneway'].fillna('no', inplace=True)
way_tags['oneway'] = way_tags['oneway'].replace('yes', True).replace('no', False).replace('-1', False).replace(-1, False).replace('alternating',False)


In [148]:
way_tags.to_file('way.geojson',driver='GeoJSON')

In [149]:
links, nodes = road.get_links_and_nodes('way.geojson', split_direction=True)
nodes = nodes.set_crs(links.crs)

In [150]:
print(len(way_tags))
print(len(links))

35259
65240


In [151]:
import networkx as nx
def main_strongly_connected_component(links, nodes=None, add_reverse=True):

    graph = nx.DiGraph()
    graph.add_edges_from(links[['a', 'b']].values.tolist())
    if 'oneway' in links.columns and add_reverse :
        graph.add_edges_from(
            links.loc[~links['oneway'].astype(bool)][['b', 'a']].values.tolist()
        )

    main_scc = None
    size = 0
    for scc in nx.strongly_connected_components(graph):
        if len(scc) > size :
            size = len(scc)
            main_scc = scc

    l = links.loc[links['a'].isin(main_scc) & links['b'].isin(main_scc)]
    if nodes is not None:
        n = nodes.loc[main_scc]
        return l, n
    return l 
    


main_links, main_nodes = main_strongly_connected_component(links, nodes, add_reverse=False)

/tmp/ipykernel_3598/4009958376.py:20: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  n = nodes.loc[main_scc]


In [152]:
print(len(main_links))

61107


In [153]:
links = main_links.copy()
nodes = main_nodes.copy()

links['length'] = links.to_crs(32618).length


links['maxspeed'] = links['maxspeed'].str.lower().str.replace('kph', '')
try:
    mph_index = links['maxspeed'].astype(str).str.lower().str.contains('mph')
    links.loc[mph_index,'maxspeed'] = links.loc[mph_index,'maxspeed'].str.lower().str.replace('mph','').astype('float')* 1.60934
except:
    print('fail to convert mph in maxspeed to float ')
    
links.loc[~links['maxspeed'].astype(str).str.isdigit(),'maxspeed']=np.nan
links['maxspeed'] = pd.to_numeric(links['maxspeed'])
speed_dict = links.dropna().groupby('highway')['maxspeed'].agg(np.mean).to_dict()
links.loc[~np.isfinite(links['maxspeed']),'maxspeed'] = links.loc[~np.isfinite(links['maxspeed']),'highway'].apply(lambda x: speed_dict.get(x))
links['time'] = links['length']/(links['maxspeed']*1000/3600)

In [154]:
links.to_file('links.geojson')
nodes.to_file('nodes.geojson')

/home/simon/anaconda3/envs/quetzal_env/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
